In [2]:
# ! pip install transformers
import transformers
print(transformers.__version__)

ModuleNotFoundError: No module named 'transformers'

In [11]:
text = "Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum \
and first released in 1991, Python's design philosophy emphasizes code \
readability with its notable use of significant whitespace."

text2 = "Gravity (from Latin gravitas, meaning 'weight'), or gravitation, is a natural phenomenon by which all \
things with mass or energy—including planets, stars, galaxies, and even light—are brought toward (or gravitate toward) \
one another. On Earth, gravity gives weight to physical objects, and the Moon's gravity causes the ocean tides. \
The gravitational attraction of the original gaseous matter present in the Universe caused it to begin coalescing \
and forming stars and caused the stars to group together into galaxies, so gravity is responsible for many of \
the large-scale structures in the Universe. Gravity has an infinite range, although its effects become increasingly \
weaker as objects get further away"

text3 = "42 is the answer to life, universe and everything."

text4 = "Forrest Gump is a 1994 American comedy-drama film directed by Robert Zemeckis and written by Eric Roth. \
It is based on the 1986 novel of the same name by Winston Groom and stars Tom Hanks, Robin Wright, Gary Sinise, \
Mykelti Williamson and Sally Field. The story depicts several decades in the life of Forrest Gump (Hanks), \
a slow-witted but kind-hearted man from Alabama who witnesses and unwittingly influences several defining \
historical events in the 20th century United States. The film differs substantially from the novel."

In [15]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer,DPRQuestionEncoder, AutoTokenizer

In [19]:
model = DPRQuestionEncoder.from_pretrained("vblagoje/dpr-question_encoder-single-lfqa-wiki")
tokenizer = AutoTokenizer.from_pretrained("vblagoje/dpr-question_encoder-single-lfqa-wiki")

input_ids = tokenizer("Why do airplanes leave contrails in the sky?", return_tensors="pt")["input_ids"]
embeddings = model(input_ids)

In [2]:
#from collections.abc import Iterable
#! pip install farm-haystack[pinecone]

In [3]:
#QG
from pipelines import pipeline
#PDF
# from PyPDF2 import PdfReader
# #torch
# import torch
# import pickle

In [26]:
# #Document store
# from haystack.document_stores import PineconeDocumentStore
from haystack import Document
# from haystack.utils import print_answers

## QA pipline

In [5]:
# import pinecone

# pinecone.init(api_key="dcb09f17-2a00-4e05-840f-eb7636bc7320", environment="us-east1-gcp")

In [6]:
# from haystack.document_stores import PineconeDocumentStore

# document_store = PineconeDocumentStore(
#     api_key='dcb09f17-2a00-4e05-840f-eb7636bc7320',
#     index='smail',
#     similarity="cosine",
#     embedding_dim=768,
#     environment="us-east1-gcp"


    
# )

In [7]:
# print(document_store.metric_type)
# print(document_store.get_document_count())
# print(document_store.get_embedding_count())

In [8]:
#Retriever
# from haystack.nodes import EmbeddingRetriever

# retriever = EmbeddingRetriever(
#     document_store=document_store,
#     embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
#     model_format="sentence_transformers"
# )

In [9]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str="Flat")

## Updating document store with docs from th given data path

In [21]:
from haystack.utils import convert_files_to_docs, fetch_archive_from_http, clean_wiki_text


# Let's first get some files that we want to use
# doc_dir = "data/tutorial12"
# s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt12.zip"
# fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

docs = [text2, text3,text4]
# Convert files to dicts
# docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
# document_store.write_documents(docs)

In [22]:
from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="vblagoje/dpr-question_encoder-single-lfqa-wiki",
    passage_embedding_model="vblagoje/dpr-ctx_encoder-single-lfqa-wiki",
)
document_store.update_embeddings(retriever)

Calling DocumentStore.update_embeddings() on an empty index


In [42]:
doc = Document(
        	content= text,
        	meta={
            	"article_title": 'article_title',
            	'section_title': 'section_title'
        	}
    	)


In [43]:
# retriever.embed_documents([doc])
embeds = retriever.embed_documents([doc])
doc.embedding = embeds[0]
document_store.write_documents([doc],duplicate_documents = 'overwrite' )
document_store.update_embeddings(retriever)

You have to provide `duplicate_documents = 'overwrite'` arg and `FAISSDocumentStore` does not support update in existing `faiss_index`.
Please call `update_embeddings` method to repopulate `faiss_index`
Writing Documents: 10000it [00:00, 424795.57it/s]       
Documents Processed: 10000 docs [00:04, 2255.16 docs/s]      


In [44]:
from haystack.utils import print_documents
from haystack.pipelines import DocumentSearchPipeline

# p_retrieval = DocumentSearchPipeline(retriever)
# res = p_retrieval.run(query="Tell me something about Arya Stark?", params={"Retriever": {"top_k": 10}})
# print_documents(res, max_text_len=512)

In [45]:
#Generator

from haystack.nodes import Seq2SeqGenerator

generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

In [46]:
#Our QA piline

from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator, retriever)

In [48]:
type(pipe)

haystack.pipelines.standard_pipelines.GenerativeQAPipeline

In [49]:
pipe.run(
    query="'What is the Latin word for gravitation?", params={"Retriever": {"top_k": 3}}
)

{'query': "'What is the Latin word for gravitation?",
 'answers': [<Answer {'answer': 'The Latin word for gravity is gravitas, meaning "weight".', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['c2ccbda03f539d216a89bfde483d2200', '2ce1de1b4d6dd8e4564795c955e0b356', '58f7f2368598f86e5772fd49758dfa93'], 'meta': {'doc_scores': [0.5461917926390552, 0.5206973302249435, 0.48826173854387944], 'content': ["Gravity (from Latin gravitas, meaning 'weight'), or gravitation, is a natural phenomenon by which all things with mass or energy—including planets, stars, galaxies, and even light—are brought toward (or gravitate toward) one another. On Earth, gravity gives weight to physical objects, and the Moon's gravity causes the ocean tides. The gravitational attraction of the original gaseous matter present in the Universe caused it to begin coalescing and forming stars and caused the stars to group together into galaxies

### Extracting Text from PDF

In [10]:
def text_from_pdf(path: str) -> str:
    reader = PdfReader(path)
    pages = reader.pages
    doc = str()
    for page in pages:
        doc = doc + page.extract_text() + " "
    return doc

In [12]:
def text_from_pdf_qg(path: str, nlp):
    reader = PdfReader(path)
    pages = reader.pages
    questions = []
    for page in pages:
        doc = page.extract_text()
        quest = nlp(doc)
        for item in quest:
            questions.append(item)
    return questions

### Updating our document store

In [13]:
def update_doc_store(doc: str, article_title : str = "None", section_title : str = "None" ):
    docs = []
    doc = Document(
            content= doc,
            meta={
                "article_title": article_title,
                'section_title': section_title
            }
        )
    docs.append(doc)
    embeds = retriever.embed_documents(docs)
    doc.embedding = embeds[0]
    document_store.write_documents(docs)

## QG

In [6]:
nlp = pipeline("e2e-qg")

In [7]:
nlp(text2)

['What is the Latin word for gravitation?',
 'What does gravity give weight to on Earth?',
 "The moon's gravity causes what?",
 'Gravity is responsible for many of the large-scale structures in what universe?']

## Testing by pdf files

In [19]:
file_name = "01.pdf"
#doc = text_from_pdf(file_name)

In [20]:
questions = text_from_pdf_qg(file_name,nlp)

In [21]:
questions

['What is the name of the group that is trading as the Taylor & Francis Group?',
 'What is a name for the company that is based in China?',
 'How many AGB equations were developed after all necessary biomass calculations were done?',
 'What is the name of the group that is widely distributed in dry forest and forest margins?',
 'Which species has many economic and ecological functions, but its environmental uses are less assessed?',
 'The best models selected have higher R2-adj and lower residual standard error?',
 'What is an important species in evergreen montane scrub?',
 'What is edible and sometimes used to extract oil?',
 'Where are leaves, twigs, and wood used to fumigate pots for milk, TELLA, and TEJ?',
 'What are the equa- tions developed to estimate biomass components of trees and shrubs in various ecosystems in the world?',
 'What can the choice of the allometric model overestimate the greenhouse gas by 40% in the Congo Basin?',
 'The site- and species-specific equation has 

In [23]:
import re
match = " my email addres is anouar.taizoukt@um6p.ma anouar@gmail.com jj"
match = re.sub(r'[\w.+-]+@[\w-]+\.[\w.-]+',"" ,match)
match

' my email addres is   jj'

In [119]:
update_doc_store(doc)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [120]:
print(document_store.get_embedding_count())

21504


In [40]:
result = pipe.run(
    query="The moon's gravity causes what?",
    params={
        "Retriever": {"top_k": 3},
        "Generator": {"top_k": 1}
    }
)

print_answers(result, details="minimum")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: The moon's gravity causes what?
Answers:
[{'answer': "The Moon's gravity causes the tides on the surface of the Earth."}]


In [47]:
answer_dic = result["answers"][0].to_dict()
answer_dic["answer"]

"The Moon's gravity causes the tides on the surface of the Earth."

In [123]:
for doc in result['documents'][1:]:
    print(doc.content, end='\n---\n')

there is a conspicuous lack of both mathematical theory and controlled experiment, as we know it. These later became decisive factors in forming modern science, and their early application came to be known as classical mechanics.
In his Elementa super demonstrationem ponderum, medieval mathematician Jordanus de Nemore introduced the concept of "positional gravity" and the use of component forces.
The first published causal explanation of the motions of planets was Johannes Kepler's Astronomia nova, published in 1609. He concluded, based on Tycho Brahe's observations on the orbit of Mars, that the planet's orbits were ellipses. This break with ancient thought was happening
---
the rotation axis of the Earth; this much smaller effect is due to the uneven distribution of the Earth's mass. To make computations feasible, scientists defined ellipsoids of revolution; a given ellipsoid would be a good compromise for measurements in a given area, such as a country or continent. The difference b